#### Questions I would like to answer
- At what time(s) of the year are crimes more likely to occur?
- What kind of crimes are prevalent? 
- Which kind of people are more likely to be criminal? What are the attributes that indicate a criminal person?

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Crimes_-_2001_to_Present.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.shape[0] - df.count() # show number of null values 

In [ ]:
districts = df['District'].unique()
districts

In [ ]:
len(districts)

there are 24 districts. i will invistigate further on these disctrics

In [ ]:
no_crimes_district = df.dropna(subset=['District']).groupby('District').count()
no_crimes_district

There is something wrong with districts 21.0 and 31.0. 
The number of crimes in these districs can't be real (too small)

In [ ]:
no_crimes_district = no_crimes_district.drop([21.0, 31.0])['ID'].sort_values()
no_crimes_district

In [ ]:
no_crimes_district.index = no_crimes_district.index.map(str)

In [ ]:
import plotly.graph_objects as go
bar_trace = go.Bar(x=no_crimes_district.index, y=no_crimes_district)
title_layout = go.Layout(title='Number of crimes per district')
figure = go.Figure(data=[bar_trace], layout=title_layout)
figure.show()

We can see the number of crimes per district, but this might not be representative as districts vary in terms of population and area.

In [ ]:
crimes_positions = df[['ID', 'X Coordinate', 'Y Coordinate']].dropna(subset=['X Coordinate', 'Y Coordinate']).reset_index(drop=True)
crimes_positions

In [ ]:
x_range = (crimes_positions['X Coordinate'].min(), crimes_positions['X Coordinate'].max())
y_range = (crimes_positions['Y Coordinate'].min(), crimes_positions['Y Coordinate'].max())

In [ ]:
x_range

In [ ]:
y_range

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=crimes_positions['X Coordinate'], y=crimes_positions['Y Coordinate'])
plt.show()


Points that are at (0,0) have wrong data. i will remove them and check again to find a pattern on places that have high rates of crimes

In [ ]:
crimes_positions = crimes_positions[crimes_positions['X Coordinate'] != 0].reset_index(drop=True)
crimes_positions.shape

In [ ]:
plt.scatter(x=crimes_positions['X Coordinate'], y=crimes_positions['Y Coordinate'], alpha=0.0059, s=0.05)
plt.show()

In [ ]:
x_range = (crimes_positions['X Coordinate'].min(), crimes_positions['X Coordinate'].max())
y_range = (crimes_positions['Y Coordinate'].min(), crimes_positions['Y Coordinate'].max())

In [ ]:
from PIL import Image
background_image = Image.open('chicago-districts.jpg')
plt.scatter(x=crimes_positions['X Coordinate'], y=crimes_positions['Y Coordinate'], alpha=0.0059, s=0.05)

plt.imshow(background_image, extent=[x_range[0], x_range[1], y_range[0], y_range[1]], aspect='auto')

plt.xlim(x_range[0], x_range[1])
plt.ylim(y_range[0], y_range[1])

plt.show()

We can see places that relatively safer in the diagram. 
Notice that crimes near district borders are way less.
The 2 districts at the southern east and at northern west are the safest.

In [ ]:
crime_type_count = df[['ID', 'Primary Type']].groupby('Primary Type')['ID'].count().sort_values(ascending=False)
crime_type_count

In [ ]:
crime_type_count_bar = go.Bar(x=crime_type_count[:10].index, y=crime_type_count[:10].values)
crime_type_count_layout = go.Layout(title='Number of Crimes Per Type', xaxis=dict(title='Crime Type'))
go.Figure(data=[crime_type_count_bar], layout=crime_type_count_layout).show()

In [ ]:
crime_type_count[:2].sum()/df.count()['ID']

In [ ]:
print(f'Number of crime types: {len(crime_type_count)-2}') # NON-CRIMINAL type is mentioned 3 times

Theft & Battery are extremly frequent. They make up 40% of 32 type of crime. if you take a deeper look you find out that there are other types of theft that are kept on their classes.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(6,24))
iter_no=0

for crime_type, _ in crime_type_count[:4].items():
    crime_type_positions = df[df['Primary Type'] == crime_type][['ID', 'X Coordinate', 'Y Coordinate']].dropna(subset=['X Coordinate', 'Y Coordinate']).reset_index(drop=True)
    axes[iter_no].scatter(x=crime_type_positions['X Coordinate'], 
                          y=crime_type_positions['Y Coordinate'], 
                          alpha=0.0059, s=0.05)
    axes[iter_no].imshow(background_image, extent=[x_range[0], x_range[1], y_range[0], y_range[1]], aspect='auto')
    axes[iter_no].set_title(f'Distribution of {crime_type.title()}')
    iter_no += 1


plt.tight_layout()
plt.show()

We can see the spread of each crime type. Based on this we can prepare stations.
for example theft is spread the most on the east border.

In [ ]:
crimes_year_trace = go.Histogram(x=df['Year'])
go.Figure(data=[crimes_year_trace], layout=go.Layout(title='Crimes Per Year')).show()

Between 2019 and 2020 the number of crimes droped by 50k. From 2020 to 2021 the number of crimes didn't change. Between 2021 to 2022 it increased by 30k bringing it back closer to the normal rate from 2014 to 2019. 
So, I would expect that the drop in 2020 was due to COVID-19 quarantine, and the increas the number of crimes in 2022 is due to the quarantine removal. Thus I expect then number of crimes to increase  slightly in the upcomming years.

In [ ]:
from datetime import datetime
crimes_month_trace = go.Histogram(x=df.sample(1_000_000)['Date'].apply(lambda d: datetime.strptime(d, '%m/%d/%Y %I:%M:%S %p')))
go.Figure(data=[crimes_month_trace], layout=go.Layout(title=f'Crimes Per Month')).show()

It's appearent from the diagram that crimes occur more on the summer months of each year (May-Sept). The occur less on winter months (Nov-Mar)